# 阶段一: 候选短语的自动化发现 (The Discovery Engine)

**总目标:** 从约17万条中国新闻中，通过三种互补的算法（Gensim Phrases, spaCy NER, spaCy Noun Chunks），大规模挖掘潜在的、有意义的多词短语，为后续的人工审核环节提供一个高质量的“候选池”。

---
### 步骤 1.0 - 环境设置与库导入

In [1]:
# --- 步骤 1.0: 环境设置与库导入  ---

# 导入必要的库
import pandas as pd
import spacy
import time
import re
import os
from gensim.models.phrases import Phrases
from collections import Counter
from tqdm.auto import tqdm # 用于显示美观的进度条
import pickle # 用于高效地保存和加载Python对象 (如列表、字典)
import multiprocessing as mp # Python内置的多进程库
import psutil # 用于智能检测CPU核心数
import shutil # 用于在服务器模式下高效复制文件

# ==============================================================================
# --- 核心配置区 (通常您只需要修改这部分) ---
# ==============================================================================

# 1. 环境配置
# 目的: 自动适配不同运行环境，解决服务器上因网络存储(NFS)导致的I/O错误
#       或本地与服务器目录结构不一致的问题。
# 选项:
#   'local': 在您自己的电脑上运行。代码将使用相对路径 (如 ../data_processed)。
#   'dsw':   在阿里云DSW或其他远程服务器上运行。代码将使用绝对路径
#            (如 /mnt/data/data_processed)，并智能地将I/O密集型操作重定向
#            到服务器本地的 /tmp 目录，以获得最佳稳定性和速度。
RUNNING_ENV = 'local'

# 2. 快速测试模式开关
# 目的: 用于快速调试代码逻辑，避免处理全量数据耗费大量时间。
# 效果:
#   True:  只处理开头指定数量(TEST_SAMPLE_SIZE)的新闻，
#          可以快速验证整个流程是否通畅。
#   False: 处理所有数据，用于正式产出最终结果。
TEST_MODE = True
TEST_SAMPLE_SIZE = 1000 # 测试模式下，从源文件读取的最大行数

# =============================================================================
# --- 路径智能管理 (根据上面的 RUNNING_ENV 自动配置，通常无需修改) ---
# =============================================================================
print(f"检测到运行环境为: 【{RUNNING_ENV.upper()}】")
TEMP_DIR = '/tmp' # 定义服务器的本地临时目录

# 根据环境定义数据根路径
if RUNNING_ENV == 'local':
    # 本地模式: 使用相对于当前脚本(通常在 'scripts' 目录)的路径
    print("使用 'local' 模式的相对路径。")
    BASE_DATA_PROCESSED_PATH = '../data_processed'
elif RUNNING_ENV == 'dsw':
    # DSW模式: 使用服务器上挂载存储的绝对路径
    print("使用 'dsw' 模式的绝对路径。")
    BASE_DATA_PROCESSED_PATH = '/mnt/data/data_processed'
    if not os.path.exists(TEMP_DIR): os.makedirs(TEMP_DIR) # 确保临时目录存在
else:
    # 错误处理，防止配置错误
    raise ValueError(f"未知的 RUNNING_ENV: '{RUNNING_ENV}'. 请选择 'local' 或 'dsw'。")

# --- 定义所有“原始”文件路径 (这些路径指向你的网络存储或项目目录) ---
INPUT_CHINA_DATA_ORIGINAL = os.path.join(BASE_DATA_PROCESSED_PATH, 'final_china_news.csv')
TOKEN_LISTS_ORIGINAL = os.path.join(BASE_DATA_PROCESSED_PATH, 'intermediate_raw_token_lists.pkl')
CANDIDATES_GENSIM_ORIGINAL = os.path.join(BASE_DATA_PROCESSED_PATH, 'candidates_gensim.pkl')
CANDIDATES_NER_ORIGINAL = os.path.join(BASE_DATA_PROCESSED_PATH, 'candidates_ner.pkl')

# --- 初始化将要在后续流程中实际使用的路径变量，默认指向原始路径 ---
INPUT_CHINA_DATA_PATH = INPUT_CHINA_DATA_ORIGINAL
TOKEN_LISTS_PATH = TOKEN_LISTS_ORIGINAL
CANDIDATES_GENSIM_PATH = CANDIDATES_GENSIM_ORIGINAL
CANDIDATES_NER_PATH = CANDIDATES_NER_ORIGINAL

# --- DSW环境下的I/O优化：重定向高负载的读写路径到 /tmp ---
if RUNNING_ENV == 'dsw':
    print("DSW 环境模式已激活，为避免I/O错误，将使用本地临时目录 /tmp ...")
    # 定义所有在本地临时目录中对应的文件路径
    TEMP_INPUT_CHINA_DATA = os.path.join(TEMP_DIR, 'final_china_news.csv')
    TEMP_TOKEN_LISTS = os.path.join(TEMP_DIR, 'intermediate_raw_token_lists.pkl')
    TEMP_CANDIDATES_GENSIM = os.path.join(TEMP_DIR, 'candidates_gensim.pkl')
    TEMP_CANDIDATES_NER = os.path.join(TEMP_DIR, 'candidates_ner.pkl')

    try:
        # 检查源文件是否存在，如果不存在，无法继续
        if not os.path.exists(INPUT_CHINA_DATA_ORIGINAL):
            raise FileNotFoundError(f"在DSW的源路径 {INPUT_CHINA_DATA_ORIGINAL} 未找到文件！")

        source_size = os.path.getsize(INPUT_CHINA_DATA_ORIGINAL)
        temp_exists = os.path.exists(TEMP_INPUT_CHINA_DATA)

        # 智能复制：仅在临时文件不存在或大小与源文件不一致时才执行复制操作，避免重复工作
        if not temp_exists or os.path.getsize(TEMP_INPUT_CHINA_DATA) != source_size:
            print(f"正在从 {INPUT_CHINA_DATA_ORIGINAL} 复制到 {TEMP_INPUT_CHINA_DATA} (首次运行可能需要几分钟)...")
            shutil.copy(INPUT_CHINA_DATA_ORIGINAL, TEMP_INPUT_CHINA_DATA)
            print("复制完成。")
        else:
            print(f"临时输入文件 {TEMP_INPUT_CHINA_DATA} 已存在且大小一致，跳过复制。")

        # [关键] 重定向路径变量，让后续代码块透明地使用/tmp下的文件
        INPUT_CHINA_DATA_PATH = TEMP_INPUT_CHINA_DATA # 预分词将读取这个临时文件
        TOKEN_LISTS_PATH = TEMP_TOKEN_LISTS           # 预分词的结果将写入这里
        CANDIDATES_GENSIM_PATH = TEMP_CANDIDATES_GENSIM # Gensim的结果将写入这里
        CANDIDATES_NER_PATH = TEMP_CANDIDATES_NER       # NER的结果将写入这里

    except Exception as e:
        print(f"❌ 处理DSW临时文件时出错: {e}")
        # 如果出错，停止执行，因为在DSW上使用原始网络路径风险很高
        raise e

# =============================================================================
# --- 全局处理与启动信息 ---
# =============================================================================

# --- 并行处理配置 ---
# 使用所有可用的物理CPU核心，并保留一个核心给操作系统，以避免系统卡顿
# logical=False 在物理机或高性能虚拟机上更稳健，避免超线程干扰
# 最多使用8个进程，防止在核心数极多的服务器上过度消耗资源
cpu_cores = psutil.cpu_count(logical=False)
N_PROCESSES = min(cpu_cores - 1 if cpu_cores > 1 else 1, 8)
if N_PROCESSES < 1: N_PROCESSES = 1

# --- 打印最终配置信息，方便追溯和调试 ---
print("\n--- 环境准备 ---")
if TEST_MODE:
    print(f"🚀🚀🚀 运行在【快速测试模式】下，将处理前 {TEST_SAMPLE_SIZE} 行数据！🚀🚀🚀")
else:
    print("🚢🚢🚢 运行在【完整数据模式】下，将处理所有数据。🚢🚢🚢")
print(f"最终输入文件路径 (读): {INPUT_CHINA_DATA_PATH}")
print(f"最终Token列表路径 (写/读): {TOKEN_LISTS_PATH}")
print(f"Gensim候选路径 (写): {CANDIDATES_GENSIM_PATH}")
print(f"NER候选路径 (写): {CANDIDATES_NER_PATH}")
print(f"将使用 {N_PROCESSES} 个进程进行并行处理。")

# --- 加载spaCy模型 ---
print("\n正在加载spaCy模型 'en_core_web_lg' (推荐使用 large 模型以获得更好的NER效果)...")
start_time = time.time()
try:
    # 'en_core_web_lg' 包含词向量，对NER等任务有帮助
    nlp = spacy.load("en_core_web_lg")
    print(f"spaCy模型 '{nlp.meta['name']}' 加载成功！耗时: {time.time() - start_time:.2f} 秒")
except OSError:
    print("错误: spaCy模型 'en_core_web_lg' 未安装。")
    print("请在你的终端或命令行中运行: python -m spacy download en_core_web_lg")
    nlp = None

检测到运行环境为: 【LOCAL】
使用 'local' 模式的相对路径。

--- 环境准备 ---
🚀🚀🚀 运行在【快速测试模式】下，将处理前 1000 行数据！🚀🚀🚀
最终输入文件路径 (读): ../data_processed\final_china_news.csv
最终Token列表路径 (写/读): ../data_processed\intermediate_raw_token_lists.pkl
Gensim候选路径 (写): ../data_processed\candidates_gensim.pkl
NER候选路径 (写): ../data_processed\candidates_ner.pkl
将使用 5 个进程进行并行处理。

正在加载spaCy模型 'en_core_web_lg' (推荐使用 large 模型以获得更好的NER效果)...
spaCy模型 'core_web_lg' 加载成功！耗时: 1.17 秒


### 步骤 1.1 - 文本预分词

需求: 为 Gensim 准备 raw_token_lists。
实现: 使用并行化的 nlp.pipe，但只启用分词功能以获得最高速度。

In [2]:
# --- 步骤 1.1: 并行预分词 ---

# 1. 加载数据
print("--- 步骤 1.1a: 数据加载 ---")
start_time_load = time.time()
df = pd.DataFrame()
try:
    read_nrows = TEST_SAMPLE_SIZE if TEST_MODE else None
    # 使用在块1中配置好的路径变量
    df = pd.read_csv(INPUT_CHINA_DATA_PATH, dtype=str, nrows=read_nrows)
    # 你的原逻辑是'CONTENT'，但你的文件列名可能是'body'等，这里假设'CONTENT'
    # 如果报错 KeyError: 'CONTENT'，请检查你的 final_china_news.csv 的列名
    df.dropna(subset=['CONTENT'], inplace=True)
    print(f"数据加载完成！耗时: {time.time() - start_time_load:.2f} 秒。 数据形状: {df.shape}")
except FileNotFoundError:
    print(f"❌ 错误: 输入文件不存在: {INPUT_CHINA_DATA_PATH}")
except KeyError:
    print(f"❌ 错误: 在 {INPUT_CHINA_DATA_PATH} 中找不到名为 'CONTENT' 的列。请检查文件内容。")


if not df.empty and nlp:
    print("\n--- 步骤 1.1b: 开始并行预分词 ---")
    start_time_process = time.time()
    raw_token_lists = []

    # 禁用所有不需要的管道组件以获得最快速度
    disabled_pipes = [pipe for pipe in nlp.pipe_names if pipe not in ['tok2vec']]

    with nlp.select_pipes(disable=disabled_pipes):
        docs = nlp.pipe(df['CONTENT'], batch_size=1000, n_process=N_PROCESSES)
        for doc in tqdm(docs, total=len(df), desc="预分词中"):
            tokens = [token.text for token in doc if not token.is_punct and not token.is_space]
            raw_token_lists.append(tokens)

    print(f"\n预分词处理完成！耗时: {(time.time() - start_time_process)/60:.2f} 分钟。")

    # 使用在块1中配置好的路径变量
    print(f"正在保存预分词结果 ({len(raw_token_lists)}篇文章) 到 {TOKEN_LISTS_PATH}...")
    with open(TOKEN_LISTS_PATH, 'wb') as f:
        pickle.dump(raw_token_lists, f)
    print("保存成功！")
else:
    print("DataFrame为空或spaCy模型未加载，跳过处理。")

--- 步骤 1.1a: 数据加载 ---
数据加载完成！耗时: 0.04 秒。 数据形状: (1000, 3)

--- 步骤 1.1b: 开始并行预分词 ---


预分词中:   0%|          | 0/1000 [00:00<?, ?it/s]


预分词处理完成！耗时: 0.50 分钟。
正在保存预分词结果 (1000篇文章) 到 ../data_processed\intermediate_raw_token_lists.pkl...
保存成功！


### 步骤 1.2 - Gensim Phrases 发现

需求: 使用上一步的 raw_token_lists 训练 Phrases 模型。
实现: 从 .pkl 文件加载数据，然后进行训练和保存。

In [4]:
# --- 步骤 1.2: Gensim Phrases 发现 (已根据正确诊断进行最终修复) ---
if os.path.exists(TOKEN_LISTS_PATH):
    print(f"--- 步骤 1.2: 开始 Gensim Phrases 发现 ---")
    print(f"正在从 {TOKEN_LISTS_PATH} 加载预分词结果...")
    with open(TOKEN_LISTS_PATH, 'rb') as f:
        raw_token_lists_gensim = pickle.load(f)
    print("加载完成。")

    print("正在为Gensim Phrases准备小写化的token列表...")
    documents_for_gensim = [[token.lower() for token in doc] for doc in tqdm(raw_token_lists_gensim, desc="转小写")]
    del raw_token_lists_gensim

    print("\n开始训练Phrases模型...")
    start_time = time.time()
    min_count = 5 if TEST_MODE else 20
    print(f"使用 min_count = {min_count}")
    # 注意：在返回字典的旧版Gensim中，分隔符通常是普通字符串
    phrases_model = Phrases(documents_for_gensim, min_count=min_count, threshold=10.0, delimiter='_')
    train_time = time.time() - start_time
    print(f"Phrases模型训练完成！耗时: {train_time:.2f} 秒。")

    print("\n正在导出Gensim发现的候选短语...")
    gensim_candidates_raw = phrases_model.export_phrases()

    gensim_candidates = {
        phrase.replace('_', ' '): score
        for phrase, score in gensim_candidates_raw.items()
    }

    print(f"Gensim Phrases发现了 {len(gensim_candidates)} 个候选短语。")

    print(f"\n正在将Gensim候选结果保存到 {CANDIDATES_GENSIM_PATH}...")
    with open(CANDIDATES_GENSIM_PATH, 'wb') as f:
        pickle.dump(gensim_candidates, f)
    print("保存成功！")
else:
    print(f"❌ 错误: Gensim的输入文件 {TOKEN_LISTS_PATH} 不存在。请先运行步骤 1.1。")

--- 步骤 1.2: 开始 Gensim Phrases 发现 ---
正在从 ../data_processed\intermediate_raw_token_lists.pkl 加载预分词结果...
加载完成。
正在为Gensim Phrases准备小写化的token列表...


转小写:   0%|          | 0/1000 [00:00<?, ?it/s]


开始训练Phrases模型...
使用 min_count = 5
Phrases模型训练完成！耗时: 0.60 秒。

正在导出Gensim发现的候选短语...
Gensim Phrases发现了 3172 个候选短语。

正在将Gensim候选结果保存到 ../data_processed\candidates_gensim.pkl...
保存成功！


### 步骤 1.3 - 命名实体识别 (NER)

需求: 从原文中并行提取命名实体。
实现: 使用 nlp.pipe 并行处理，只启用 NER 相关组件。

In [5]:
# --- 步骤 1.3: 并行命名实体识别 (NER) ---

if 'df' in locals() and not df.empty and nlp:
    print("\n--- 步骤 1.3: 开始并行NER提取 ---")
    start_time_process = time.time()
    ner_candidates = Counter()

    print(f"将使用 {N_PROCESSES} 个进程。")
    TARGET_ENTITY_LABELS = {'ORG', 'PERSON', 'GPE', 'NORP', 'FAC', 'LOC', 'PRODUCT', 'EVENT'}

    # 专注于 NER, 启用其依赖的组件
    with nlp.select_pipes(enable=["tok2vec", "tagger", "ner"]):
        docs = nlp.pipe(df['CONTENT'], batch_size=500, n_process=N_PROCESSES)
        for doc in tqdm(docs, total=len(df), desc="NER提取中"):
            for ent in doc.ents:
                if ent.label_ in TARGET_ENTITY_LABELS and ' ' in ent.text:
                    ner_candidates[ent.text.strip().lower()] += 1

    print(f"\nNER提取完成！耗时: {(time.time() - start_time_process)/60:.2f} 分钟。")

    print(f"正在保存NER候选结果 ({len(ner_candidates)}个) 到 {CANDIDATES_NER_PATH}...")
    with open(CANDIDATES_NER_PATH, 'wb') as f:
        pickle.dump(ner_candidates, f)
    print("保存成功！")

    # 所有任务完成，清理DataFrame
    del ner_candidates
    del df
else:
    print("DataFrame (df) 未加载或为空，跳过NER提取。")

print("\n\n✅✅✅ 阶段一（精简版）全部完成！✅✅✅")
print("Gensim 和 NER 两种来源的候选短语都已生成并保存。")
print("下一步是运行后续的Notebook来整合和审核这些结果。")


--- 步骤 1.3: 开始并行NER提取 ---
将使用 5 个进程。


NER提取中:   0%|          | 0/1000 [00:00<?, ?it/s]


NER提取完成！耗时: 0.55 分钟。
正在保存NER候选结果 (7920个) 到 ../data_processed\candidates_ner.pkl...
保存成功！


✅✅✅ 阶段一（精简版）全部完成！✅✅✅
Gensim 和 NER 两种来源的候选短语都已生成并保存。
下一步是运行后续的Notebook来整合和审核这些结果。
